In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

page = requests.get(" https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page.status_code
page.content

soup = BeautifulSoup(page.content, 'lxml')
  
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
#print(df[0].to_json(orient='records'))
df = df[0]
df.head()


,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [2]:
##cleaning data - removing Not assigned boroughs and replacing not assigned neighbourhoods with boroughs.
df.loc[0,:]
df.columns = df.loc[0,:]#['a', 'b']
df.drop([0],axis=0,inplace = True)
df = df[df['Borough']!='Not assigned']  # removing entries with no value for Borough
df = df.reset_index()                   #resetting index after removing
df.drop(['index'],axis=1,inplace = True)
temp = df[df['Neighbourhood']=='Not assigned'].index.tolist() # getting a list of Neighbourhoods with the value "Not assigned."
df.loc[temp,'Neighbourhood'] = df.loc[temp,'Borough'] ## replacing those neighbourhoods with borough name
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [3]:
df.shape




(212, 3)